In [3]:
import os
from PIL import Image

# A 폴더와 B 폴더 경로 설정
A_folder_path = '/content/local/A_folder/'
B_folder_path = '/content/local/B_folder/'

# 폴더에서 이미지 파일 목록 불러오기
A_image_files = [f for f in os.listdir(A_folder_path) if f.endswith('.jpg') or f.endswith('.png')]
B_image_files = [f for f in os.listdir(B_folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# 이미지 불러오기 (예시로 A 폴더의 첫 번째 이미지)
example_A_image = Image.open(os.path.join(A_folder_path, A_image_files[0]))

# 이미지 확인 (옵션)
#example_A_image.show()


In [4]:
# 리사이징할 이미지의 크기 설정
resize_shape = (128, 128)

# A 폴더의 모든 이미지 리사이징
A_images_resized = [Image.open(os.path.join(A_folder_path, f)).resize(resize_shape) for f in A_image_files]

# B 폴더의 모든 이미지 리사이징
B_images_resized = [Image.open(os.path.join(B_folder_path, f)).resize(resize_shape) for f in B_image_files]


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# 이미지를 numpy 배열로 변환
A_images_array = np.array([img_to_array(img) for img in A_images_resized])
B_images_array = np.array([img_to_array(img) for img in B_images_resized])

# 데이터 정규화 (0~255 -> 0~1)
A_images_array = A_images_array.astype('float32') / 255.
B_images_array = B_images_array.astype('float32') / 255.


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# 오토인코더 모델 구성
input_img = Input(shape=(128, 128, 3))

# 인코더 부분
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# # 디코더 부분
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
# x = UpSampling2D((2, 2))(x)
# x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
# decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# 오토인코더 모델 생성
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# 모델 학습 (A 폴더와 B 폴더의 이미지를 모두 사용)
autoencoder.fit(np.concatenate([A_images_array, B_images_array]),
                np.concatenate([A_images_array, B_images_array]),
                epochs=50, batch_size=128, shuffle=True)


Epoch 1/50
39/39 [==============================] - 21s 146ms/step - loss: 0.6111
Epoch 2/50
39/39 [==============================] - 5s 133ms/step - loss: 0.5514
Epoch 3/50
39/39 [==============================] - 5s 126ms/step - loss: 0.5455
Epoch 4/50
39/39 [==============================] - 5s 124ms/step - loss: 0.5425
Epoch 5/50
39/39 [==============================] - 5s 131ms/step - loss: 0.5403
Epoch 6/50
39/39 [==============================] - 5s 125ms/step - loss: 0.5393
Epoch 7/50
39/39 [==============================] - 5s 125ms/step - loss: 0.5379
Epoch 8/50
39/39 [==============================] - 5s 132ms/step - loss: 0.5393
Epoch 9/50
39/39 [==============================] - 5s 127ms/step - loss: 0.5370
Epoch 10/50
39/39 [==============================] - 5s 129ms/step - loss: 0.5372
Epoch 11/50
39/39 [==============================] - 5s 133ms/step - loss: 0.5364
Epoch 12/50
39/39 [==============================] - 5s 128ms/step - loss: 0.5365
Epoch 13/50
39/39 [=====

In [8]:
# 인코더 모델 생성
encoder = Model(inputs=autoencoder.input, outputs=encoded)

# A, B 폴더의 이미지에 대한 잠재 벡터 추출
A_latent_vectors = encoder.predict(A_images_array)
B_latent_vectors = encoder.predict(B_images_array)

# 잠재 벡터의 모양을 1차원으로 변경 (옵션)
A_latent_vectors = A_latent_vectors.reshape((A_latent_vectors.shape[0], -1))
B_latent_vectors = B_latent_vectors.reshape((B_latent_vectors.shape[0], -1))


83/83 [==============================] - 0s 4ms/step


In [11]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# 계산할 k의 범위
K = range(10,20 )

# 각 k별로 모델을 학습시키고 inertia 값을 저장
inertia = []
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(A_latent_vectors)
    inertia.append(kmeans.inertia_)

# Elbow 그래프를 그린다
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('The Elbow Method')
plt.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [12]:
from sklearn.cluster import KMeans

# 팔꿈치 지점을 찾는 코드 (자동화 되지 않았으며, 그래프를 보고 판단해야 함)
elbow_k = 20  # 여기에 팔꿈치 지점에서의 k 값을 입력하세요

# K-means 알고리즘으로 클러스터링
# elbow_k를 n_clusters로 사용
kmeans_A = KMeans(n_clusters=elbow_k, random_state=0).fit(A_latent_vectors)
kmeans_B = KMeans(n_clusters=elbow_k, random_state=0).fit(B_latent_vectors)

# A, B 폴더의 이미지에 대한 클러스터 라벨 추출
A_cluster_labels = kmeans_A.labels_
B_cluster_labels = kmeans_B.labels_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [15]:
import pandas as pd
# A 폴더의 특정 상품 이미지 선택 (예: 첫 번째 이미지)
selected_A_index = 0
selected_A_cluster = A_cluster_labels[selected_A_index]

# 선택된 상품과 동일한 클러스터에 속한 B 폴더의 상품 찾기
similar_B_indices = np.where(B_cluster_labels == selected_A_cluster)[0]

# 유사한 상품의 이미지 목록 출력
for idx in similar_B_indices:
    similar_image = B_images_resized[idx]
    similar_image.show()  # 이미지 출력 (옵션)

# 유사한 상품의 파일명 목록 작성
similar_B_filenames = [B_image_files[idx] for idx in similar_B_indices]

# DataFrame으로 변환
df = pd.DataFrame(similar_B_filenames, columns=['Similar_Products'])

# CSV 파일로 저장
csv_file_path = '/content/drive/MyDrive/K-fashion/vector model/B_result/similar_products.csv'  # 저장할 경로를 지정해주세요.
df.to_csv(csv_file_path, index=False)
